# Ensemble Learning

## Initial Imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [4]:
pwd

'/Users/coneychiwabee/Homework_drafts_TRF to next folder/11_Classification'

## Read the CSV and Perform Basic Data Cleaning

In [5]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [6]:
d = list(df.columns.values.tolist())

In [7]:
len(d)

86

## Split the Data into Training and Testing

In [8]:
# Create our features
X = df.drop(['loan_status'], axis=1)

# Create our target
y = df['loan_status']

In [9]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.052138,2.219423,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.390633,1.897432,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.000000,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,2.000000,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,3.000000,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,18.000000,19.000000,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06


In [10]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

Ecoding the Independent Varible

In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), ['home_ownership', 'verification_status','issue_d','pymnt_plan', 'initial_list_status', 'next_pymnt_d','application_type', 'hardship_flag', 'debt_settlement_flag'])], remainder='passthrough')

X = ct.fit_transform(X) #T

In [12]:
print(X[0])

[0.000000e+00 0.000000e+00 0.000000e+00 1.000000e+00 0.000000e+00
 1.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 1.000000e+00
 1.000000e+00 0.000000e+00 1.000000e+00 0.000000e+00 1.000000e+00
 1.000000e+00 0.000000e+00 1.000000e+00 1.000000e+00 1.050000e+04
 1.719000e-01 3.753500e+02 6.600000e+04 2.724000e+01 0.000000e+00
 0.000000e+00 8.000000e+00 0.000000e+00 1.609000e+03 1.400000e+01
 1.020487e+04 1.020487e+04 3.552900e+02 3.552900e+02 2.951300e+02
 6.016000e+01 0.000000e+00 0.000000e+00 0.000000e+00 3.753500e+02
 0.000000e+00 1.000000e+00 0.000000e+00 1.780000e+02 3.819900e+04
 3.000000e+00 4.000000e+00 2.000000e+00 4.000000e+00 3.000000e+00
 3.659000e+04 5.900000e+01 1.000000e+00 2.000000e+00 1.553000e+03
 5.800000e+01 3.700000e+03 1.000000e+00 0.000000e+00 4.000000e+00
 6.000000e+00 4.775000e+03 4.470000e+02 7.770000e+01 0.000000e+00
 0.000000e+00 9.300000e+01 9.600000e+01 3.000000e+00 3.000000e+00
 0.000000e+00 2.000000e+01 1.000000e+00 0.000000e+00 1.000000e+00
 2.000000e

Encoding the Dependent Variable

In [13]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [14]:
print(y)

[1 1 1 ... 1 1 1]


In [15]:
# Split the X and y into X_train, X_test, y_train, y_test
# YOUR CODE HERE

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 1)

In [16]:
print(X_train)

[[0.00000e+00 0.00000e+00 1.00000e+00 ... 1.42920e+04 5.83000e+04
  1.50000e+04]
 [0.00000e+00 0.00000e+00 1.00000e+00 ... 5.25870e+04 2.45000e+04
  4.04920e+04]
 [0.00000e+00 1.00000e+00 0.00000e+00 ... 7.89440e+04 6.54000e+04
  8.06420e+04]
 ...
 [0.00000e+00 1.00000e+00 0.00000e+00 ... 7.64100e+03 1.37000e+04
  1.20000e+04]
 [0.00000e+00 0.00000e+00 0.00000e+00 ... 7.96500e+04 2.27000e+04
  6.69410e+04]
 [0.00000e+00 1.00000e+00 0.00000e+00 ... 2.00731e+05 6.21000e+04
  1.99166e+05]]


In [17]:
print(X_test)

[[0.0000e+00 0.0000e+00 0.0000e+00 ... 3.1265e+04 6.9000e+03 5.5850e+04]
 [0.0000e+00 1.0000e+00 0.0000e+00 ... 3.2199e+04 2.7400e+04 3.6550e+04]
 [0.0000e+00 1.0000e+00 0.0000e+00 ... 6.4318e+04 2.3800e+04 5.9638e+04]
 ...
 [0.0000e+00 1.0000e+00 0.0000e+00 ... 1.6351e+04 6.0000e+03 1.5000e+04]
 [0.0000e+00 0.0000e+00 0.0000e+00 ... 1.5682e+04 2.5000e+03 1.5524e+04]
 [0.0000e+00 0.0000e+00 0.0000e+00 ... 3.3444e+04 1.7600e+04 5.8738e+04]]


In [18]:
print(y_train)

[1 1 1 ... 1 1 1]


In [19]:
print(y_test)

[1 1 1 ... 1 1 1]


## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [20]:
# Create the StandardScaler instance.
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [21]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_train[:,19:] = sc.fit_transform(X_train[:,19:])

In [22]:
# Scale the training and testing data
X_test[:,19:] = sc.transform(X_test[:,19:])

In [23]:
print(X_train.dtype)

float64


In [24]:
print(y_train.dtype)

int64


## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [25]:
# Resample the training data with the BalancedRandomForestClassifier

from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.datasets import make_classification
brfc = BalancedRandomForestClassifier(random_state=1)
brfc.fit(X_train,y_train)



BalancedRandomForestClassifier(random_state=1)

In [26]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = brfc.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.7785287327844104

In [27]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[   69,    32],
       [ 2157, 14947]])

In [28]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.03      0.68      0.87      0.06      0.77      0.59       101
          1       1.00      0.87      0.68      0.93      0.77      0.61     17104

avg / total       0.99      0.87      0.68      0.93      0.77      0.61     17205



In [29]:
# List the features sorted in descending order by feature importance
important_features_dict = {}
for idx, val in enumerate(brfc.feature_importances_):
    important_features_dict[idx] = val

important_features_list = sorted(important_features_dict,
                                 key=important_features_dict.get,
                                 reverse=True)

print(important_features_list)

[34, 32, 35, 39, 33, 20, 8, 31, 21, 72, 23, 54, 9, 94, 56, 62, 44, 30, 28, 19, 63, 49, 92, 91, 29, 67, 22, 80, 50, 51, 66, 93, 61, 55, 59, 69, 68, 78, 71, 77, 81, 26, 87, 60, 74, 57, 79, 70, 48, 82, 75, 58, 88, 53, 45, 86, 52, 13, 7, 76, 46, 36, 43, 47, 25, 73, 14, 24, 5, 2, 89, 3, 16, 12, 4, 1, 6, 15, 11, 27, 85, 0, 64, 40, 10, 17, 18, 37, 38, 41, 42, 65, 83, 84, 90]


### Easy Ensemble Classifier

In [30]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
model = EasyEnsembleClassifier(n_estimators=10)
model.fit(X_train, y_train)
# cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# scores = cross_val_score(model, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)

EasyEnsembleClassifier()

In [31]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9234354884272338

In [32]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[   92,     9],
       [ 1095, 16009]])

In [33]:
# Print the imbalanced classification report

print(classification_report_imbalanced(y_test, y_pred))


                   pre       rec       spe        f1       geo       iba       sup

          0       0.08      0.91      0.94      0.14      0.92      0.85       101
          1       1.00      0.94      0.91      0.97      0.92      0.85     17104

avg / total       0.99      0.94      0.91      0.96      0.92      0.85     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    EasyEnsembleClassifier 

2. Which model had the best recall score?

    EasyEnsembleClassifier

3. Which model had the best geometric mean score?

   EasyEnsembleClassifier

4. What are the top three features?

   total_pymnt_inv, out_prncp_inv, total_rec_prncp